In [5]:
import spacy
import langdetect
import glob
nlp = spacy.load('en')
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import re
from gensim.models.phrases import Phrases,Phraser
from nltk.corpus import stopwords
stopwords = list(set(stopwords.words('english')))
import random
from modules import utils

In [2]:
crawled = pd.read_csv("data/crawl_data(7379 pages - employment-social-development).csv").drop_duplicates(subset=["text"], keep="first")
crawled["lang"] = crawled.text.apply(lambda x: langdetect.detect(str(x)))
crawled = crawled[crawled["lang"] == "en"].text;len(crawled)

2941

In [22]:
crawled.head()

0    Employment  and Social Development Canada (ESD...
1    The Government of Canada is prepared to suppor...
2    The Government of Canada launches funding oppo...
3    Minister Duclos releases report on nationwide ...
4    Hire a temporary worker through the Temporary ...
Name: text, dtype: object

In [23]:
#program_files = glob.glob('data/program_descriptions/*.csv')
#programs = pd.concat([pd.read_csv(file) for file in program_files]).reset_index()[["Program","Description"]]
#len(programs)

In [24]:
#programs.tail()

In [25]:
crawled_corpus = crawled.str.cat(sep=" ")
#programs_corpus = programs.Description.str.cat(sep=" ")
corpus = crawled_corpus# + programs_corpus

In [26]:
corpus[:4000]

"Employment  and Social Development Canada (ESDC) works to improve the standard of living  and quality of life for all Canadians. We do this by promoting a labour force  that is highly skilled. We also promote an efficient and inclusive labour  market. The Government of Canada is ready to support workers and their families who are affected by the April 24, 2017, U.S. decision to impose duties on Canadian softwood lumber products. Tab 1: Enabling Accessibility Fund: mid-sized projects Tab 2: Help design the new Canada Service Corps program Tab 3: See what Canadians had to say about reducing poverty Help design the new Canada Service Corps program See what Canadians had to say about reducing poverty Benefits, Canada EI Commission, Wage Earners Protection Program, and economic regions. Payment dates for recurring Government of Canada benefit payments. Job opportunities, work permits, Social Insurance Number, criminal record checks and security clearances. Canada Pension Plan, Old Age Secu

In [45]:
corpus_clean = re.sub('[^a-zA-Z0-9\s\.]+', '', corpus).lower()
corpus_clean = re.sub("\s{2,}"," ",corpus_clean)

In [46]:
corpus_clean[:800]

'employment and social development canada esdc works to improve the standard of living and quality of life for all canadians. we do this by promoting a labour force that is highly skilled. we also promote an efficient and inclusive labour market. the government of canada is ready to support workers and their families who are affected by the april 24 2017 u.s. decision to impose duties on canadian softwood lumber products. tab 1 enabling accessibility fund midsized projects tab 2 help design the new canada service corps program tab 3 see what canadians had to say about reducing poverty help design the new canada service corps program see what canadians had to say about reducing poverty benefits canada ei commission wage earners protection program and economic regions. payment dates for recur'

In [47]:
sents = sent_tokenize(corpus_clean)
sents = list(set(sents))
sents = [re.sub('[^a-zA-Z0-9\s]+', '', sent) for sent in sents];len(sents)

148188

In [49]:
sents[1]

'the submitters say that the failed to provide a fair process for the registration of the new unions and failed to protect union supporters from discrimination and intimidation by company officials'

In [50]:
#sents_clean = [re.sub('[^a-zA-Z0-9\s]+', '', sent).lower() for sent in sents];sents_clean[0]

In [51]:
sents_stream = [sent.split() for sent in sents];sents_stream[1]

['the',
 'submitters',
 'say',
 'that',
 'the',
 'failed',
 'to',
 'provide',
 'a',
 'fair',
 'process',
 'for',
 'the',
 'registration',
 'of',
 'the',
 'new',
 'unions',
 'and',
 'failed',
 'to',
 'protect',
 'union',
 'supporters',
 'from',
 'discrimination',
 'and',
 'intimidation',
 'by',
 'company',
 'officials']

## Generate N-Grams From Co-Occuring Terms

"default" scoring: <i>from “Efficient Estimaton of Word Representations in Vector Space” by
Mikolov, et. al.: (count(worda followed by wordb) - min_count) * N / (count(worda) * count(wordb)) > threshold`, where N is the total vocabulary size.</i>


"npmi" scoring: <i>normalized pointwise mutual information, from “Normalized (Pointwise) Mutual
Information in Colocation Extraction” by Gerlof Bouma: ln(prop(worda followed by wordb) / (prop(worda)*prop(wordb))) / - ln(prop(worda followed by wordb) where prop(n) is the count of n / the count of everything in the entire corpus.</i>

In [52]:
def generate_n_gram_transformers(stream,n_gram = 3,scoring="default",min_count=5,threshold=10,common_terms=None):
    streams = [stream]    
    grams = [stream]
    for n in range(1,n_gram):
        gram = Phraser(Phrases(streams[-1],scoring=scoring,min_count=min_count,threshold=threshold,common_terms=common_terms))
        streams.append(list(gram[streams[-1]]))
        grams.append(gram)
        
    return grams
        

In [53]:
_,to_bigrams,to_trigrams,to_quadgrams = generate_n_gram_transformers(sents_stream,n_gram=4,
                                                   scoring="default",min_count=30,
                                                   threshold=10,common_terms=stopwords)

In [54]:
quad_stream = list(to_quadgrams[to_trigrams[to_bigrams[sents_stream]]])
tri_stream = list(to_trigrams[to_bigrams[sents_stream]])
quad_sents = [' '.join(sent) for sent in quad_stream]
tri_sents = [' '.join(sent) for sent in tri_stream]

In [66]:
quad_stream[random.randint(0,len(quad_stream))]

['the',
 'basic_needs',
 'amount',
 'helps',
 'with',
 'the',
 'cost',
 'of',
 'food',
 'clothing',
 'transportation',
 'and',
 'necessary',
 'personal',
 'items',
 'for',
 'the',
 'recipient',
 'a',
 'spouse',
 'and',
 'any',
 'dependent_children',
 'or',
 'dependent',
 'adults',
 'if',
 'applicable']

## Word2Vec

In [67]:
from gensim.models.word2vec import Word2Vec

In [131]:
WORD2VEC_EMBEDDING_DIM = 50

In [106]:
model = Word2Vec(quad_stream, size=WORD2VEC_EMBEDDING_DIM, window=12, min_count=30, workers=4)

In [107]:
model.wv.most_similar("visa")

[('permanent_resident', 0.7319332957267761),
 ('work_permits', 0.7219541072845459),
 ('foreign', 0.7193650603294373),
 ('canadian_citizens', 0.7176936864852905),
 ('province_of_quebec', 0.7137880325317383),
 ('permanent_residence', 0.7039188742637634),
 ('certificate', 0.6946704983711243),
 ('hire_a_temporary', 0.6796412467956543),
 ('foreign_nationals', 0.6738460063934326),
 ('work_permit', 0.6542705297470093)]

In [114]:
model.wv.save_word2vec_format("data/word2vec_esdc.vec")

In [10]:
pretrained_embeddings = utils.load_embedding_model("data/wiki-news-300d-1M-subword.vec")

In [133]:
len(model.wv.vocab.keys()),len(pretrained_embeddings)

(10545, 999995)

## Sidecar Approach  -- Concatenate the Trained Embeddings to the Pretrained Embeddings 

In [138]:
PRETRAINED_DIM = 300
CONCAT_DIM = PRETRAINED_DIM+WORD2VEC_EMBEDDING_DIM
concat_embeddings = {}
for key,vec in pretrained_embeddings.items():
    if key in model.wv.vocab:
        concat_embeddings[key] = np.hstack((vec,model.wv[key]))
    else:
        concat_embeddings[key] = np.hstack((vec,np.zeros(WORD2VEC_EMBEDDING_DIM)))
for key in model.wv.vocab.keys():
    if key not in concat_embeddings:
        concat_embeddings[key] = np.hstack((np.zeros(PRETRAINED_DIM),model.wv[key]))

In [143]:
"hockey" in concat_embeddings,"service_canada" in concat_embeddings,

(True, True)

In [145]:
concat_embeddings["service_canada"]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  